In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## import dataset

In [2]:
dataset = pd.read_csv('employee.csv')
dataset = dataset.iloc[:, 1:126]

# tX = dataset.iloc[:, 2:124]
# tY = dataset.iloc[:, 125]
# X = dataset.iloc[:, 2:123].values #idや名前は除去(特徴量ではない)
# y = dataset.iloc[:, 125].values #output(0 or 1) 

In [3]:
dataset.head()

,GAME,COUNTRY,YEAR,NAPFLT,GOBFLT,SERAMY,NWYAMY,GASAMY,AEGFLT,TUNAMY,...,MARFLT,ROMFLT,EDIAMY,EDIFLT,LONAMY,NAPAMY,NAOFLT,MUNAMY,LONFLT,OUTPUT
0,1.0,RUS,1901,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],4.0
1,1.0,AUS,1901,[],[],AUS,[],[],[],[],...,[],[],[],[],[],[],[],[],[],4.0
2,1.0,GER,1901,[],[],AUS,[],[],[],[],...,[],[],[],[],[],[],[],[],[],4.0
3,1.0,TUR,1901,[],[],AUS,[],[],[],[],...,[],[],[],[],[],[],[],[],[],4.0
4,1.0,FRA,1901,[],[],AUS,[],[],[],[],...,[],[],[],[],[],[],[],FRA,[],6.0


国別に分けて, outputのデータを最終年の補給地の数にする

In [4]:
rus_dataset = dataset[dataset.COUNTRY == 'RUS']
aus_dataset = dataset[dataset.COUNTRY == 'AUS']
ger_dataset = dataset[dataset.COUNTRY == 'GER']
tur_dataset = dataset[dataset.COUNTRY == 'TUR']
fra_dataset = dataset[dataset.COUNTRY == 'FRA']
eng_dataset = dataset[dataset.COUNTRY == 'ENG']
ita_dataset = dataset[dataset.COUNTRY == 'ITA']

In [5]:
eng_dataset.head()

,GAME,COUNTRY,YEAR,NAPFLT,GOBFLT,SERAMY,NWYAMY,GASAMY,AEGFLT,TUNAMY,...,MARFLT,ROMFLT,EDIAMY,EDIFLT,LONAMY,NAPAMY,NAOFLT,MUNAMY,LONFLT,OUTPUT
5,1.0,ENG,1901,[],[],AUS,[],[],[],[],...,[],[],[],[],[],[],[],FRA,[],3.0
12,1.0,ENG,1902,[],RUS,AUS,[],[],TUR,[],...,[],[],[],[],[],[],[],FRA,[],4.0
19,1.0,ENG,1903,[],RUS,AUS,[],[],TUR,[],...,[],[],[],[],[],[],[],FRA,[],4.0
26,1.0,ENG,1904,[],RUS,AUS,[],FRA,TUR,[],...,[],[],[],[],ENG,[],[],FRA,[],3.0
33,1.0,ENG,1905,FRA,RUS,AUS,[],FRA,TUR,[],...,[],[],[],[],ENG,[],[],FRA,[],3.0


In [6]:
from sklearn.utils import shuffle

In [7]:
tX = eng_dataset.iloc[:, 0:124]
tX = shuffle(tX)
tX = tX.drop_duplicates('GAME')
tX = tX.sort_values('GAME').iloc[:, 2:124]
tY = eng_dataset.drop_duplicates('GAME', keep = 'last').iloc[:,124]

In [8]:
tX.head()

,YEAR,NAPFLT,GOBFLT,SERAMY,NWYAMY,GASAMY,AEGFLT,TUNAMY,NWYFLT,GASFLT,...,GALAMY,MARFLT,ROMFLT,EDIAMY,EDIFLT,LONAMY,NAPAMY,NAOFLT,MUNAMY,LONFLT
47,1907,FRA,RUS,AUS,[],FRA,TUR,[],RUS,[],...,GER,[],[],[],[],ENG,[],[],FRA,[]
82,1905,AUS,RUS,TUR,[],FRA,TUR,[],ENG,[],...,RUS,[],[],[],[],ENG,[],ENG,GER,ENG
215,1909,ITA,RUS,TUR,RUS,FRA,TUR,[],ENG,[],...,RUS,[],ITA,ENG,ENG,ENG,ITA,ENG,RUS,FRA
320,1913,ITA,RUS,AUS,RUS,FRA,TUR,[],ENG,ITA,...,AUS,ITA,ITA,ENG,ENG,ENG,ITA,FRA,AUS,ENG
411,1910,ITA,RUS,RUS,RUS,FRA,TUR,[],ENG,ITA,...,RUS,ITA,ITA,ENG,ENG,ENG,ITA,FRA,GER,ENG


In [9]:
tY.head()

47      3.0
152     4.0
229     5.0
341    10.0
460     4.0
Name: OUTPUT, dtype: float64

In [10]:
print(tX.shape,tY.shape)

(1589, 122) (1589,)


In [11]:
X = tX.values
y = tY.values

## Encoding categorical data
データの成形を行う

In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

##labelをエンコードする
le = LabelEncoder()
le.fit(['RUS','AUS','GER','TUR','FRA','ENG', 'ITA', '[]'])

for i in range(121):
    X[:, i+1] = le.transform(X[:, i+1])

### Gender
# labelencoder_X_2 = LabelEncoder()
# X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2]) 

In [13]:
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()

In [14]:
from sklearn import preprocessing

In [15]:
X_normalized = preprocessing.normalize(X, norm='l2')
X_normalized = X_normalized[:, 1:]
print(X_normalized)
# X = X[:, 1:]

[[ 0.          0.00052414  0.         ...,  0.00366898  0.00104828
   0.00366898]
 [ 0.          0.          0.         ...,  0.00052474  0.00157422
   0.00052474]
 [ 0.          0.          0.00052366 ...,  0.00052366  0.0026183
   0.00104732]
 ..., 
 [ 0.          0.00052425  0.         ...,  0.00052425  0.00104851
   0.00052425]
 [ 0.          0.00052508  0.         ...,  0.00052508  0.00157525
   0.00052508]
 [ 0.          0.          0.00052426 ...,  0.00052426  0.00157277
   0.00052426]]


##  データセットと学習セットに分ける

In [16]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# モデルの作成

In [39]:
from sklearn import svm
# from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

In [ ]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
reg = GridSearchCV(svm.SVR(), tuned_parameters, cv=5, scoring='mean_squared_error')
reg.fit(X_train, y_train)

# サポートベクトル回帰を学習データ使って作成
# reg = svm.SVR(kernel='rbf', C=10).fit(X_train, y_train)

# テストデータに対する予測結果のPLOT
plt.plot(X_test, y_test, 'bo-', X_test, reg.predict(X_test), 'ro-')
plt.show()

# print(reg.score(X_test, y_test))

/Users/tela/.anyenv/envs/pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/tela/.anyenv/envs/pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/tela/.anyenv/envs/pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/tela/.anyenv/envs/pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
predict = reg.predict(X_test)
print(sqrt(mean_squared_error(y_test, predict)))